### Import models 

In [8]:
import torch
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import torchvision.models as models
import torch.optim as optim

In [100]:
BATCH_SIZE = 16

transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=T.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [101]:
img = next(iter(trainloader))[0][0]
print(img.size())

torch.Size([3, 32, 32])


In [98]:
def show_sample_imgs():
    fig = plt.figure()
    i = 0 
    for x,target in trainloader:
        img = x[0].numpy()
        img = np.transpose(img, (1, 2, 0))
        cls = classes[target[0]]
        ax = fig.add_subplot(2,2,(i+1))
        ax.imshow(img)
        ax.set_title(cls)
        i+=1
        if(i == 4):
            break 
    plt.show()

In [99]:
show_sample_imgs()

ValueError: Floating point image RGB values must be in the 0..1 range.

### Load pretrained resnet model 

In [41]:
model = models.vgg16(pretrained=True)

In [42]:
for param in model.parameters():
    param.requires_grad = False

In [43]:
model.classifier._modules['6'] = nn.Linear(4096, 10)

In [44]:
for param in model.classifier[6].parameters():
    param.requires_grad = True

In [45]:
print(model.classifier)

Sequential (
  (0): Linear (25088 -> 4096)
  (1): ReLU (inplace)
  (2): Dropout (p = 0.5)
  (3): Linear (4096 -> 4096)
  (4): ReLU (inplace)
  (5): Dropout (p = 0.5)
  (6): Linear (4096 -> 10)
)


In [52]:
model = model.cuda()

### Preprocess input 

In [93]:
def preprocess(img, size=(3,16,16)):
    transform = T.Compose([
        T.Scale(size),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(img)

In [94]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

In [95]:
img = next(iter(trainloader))[0][0]
img = Variable(img)
T.scale

AssertionError: 

### Forward pass resnet model 

In [69]:
def train(model,solver,epochs = 3):
    
    #loss function 
    loss = nn.CrossEntropyLoss()
    
    #variable to save the loss of each epoch 
    training_loss = []
    l = 0 
    
    #iterate over the epochs 
    for i in range(0,epochs):
        
        #iterate over the training set 
        for x,target in trainloader:
            #zero out the gradient first 
            solver.zero_grad()
            
            #convert the input set to a cuda variable to run on gpu 
            data = Variable(x).cuda()
            trgt = Variable(target).cuda()
            
            print(data.size())
            data = preprocess(data)
            #forward pass
            y = model(data)
            
            #error calculation 
            outpt= loss(y,trgt)
            
            #calculate the gradient 
            outpt.backward()
            
            #update the parameters 
            solver.step()

            # the current loss 
            l = outpt.cpu().data.numpy()[0]
        training_loss.append(l)
    
    #plot the loss of each epoch run 
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    epoch = list(range(1, 1+len(training_loss)))
    t_line, = plt.plot(epoch, training_loss, 'co-', label='Training Loss')
    plt.legend(handles=[t_line])
    plt.show()
    
    return model 

### Extract maximum score 

In [70]:
trained_model = train(model,optimizer)

torch.Size([16, 3, 32, 32])


RuntimeError: requested resize to (224, 224)x2 ((224, 224, 224, 224) elements in total), but the given tensor has a size of 16x3x32x32 (49152 elements). autograd's resize can only change the shape of a given tensor, while preserving the number of elements. 

In [61]:
x = Variable(torch.Tensor(2,3))

In [62]:
x.resize

<bound method Variable.resize of Variable containing:
1.00000e-40 *
 -4.6903  0.4557 -4.6903
  0.4557  0.0004  0.0000
[torch.FloatTensor of size 2x3]
>